In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from descriptors_aggregator import Aggregator
import pandas as pd
from tqdm import tqdm

from rdkit import rdBase
blocker = rdBase.BlockLogs()

tqdm.pandas()


In [ ]:
data = pd.read_csv('cleared_pubchem10m.csv')
data.head()

### Splitting molecules in substructures

In [ ]:
from splitter import Splitter

agg = Aggregator()

splitter = Splitter()
data['substructures'] = data['smiles'].progress_apply(lambda smi: splitter.get_substructures_smiles_and_merged(smi))

data = data.reset_index(drop = True)

data = data.dropna(subset=['substructures'])
data = data.reset_index(drop = True)

### Dropping bad molecules

In [ ]:
dropping = set()
for i in range(len(data['substructures'])):
    for j in data['substructures'].iloc[i]:
        if j != '$':
            if j is None:
                dropping.add(i)
data = data.drop(index = dropping).reset_index(drop = True)

In [ ]:
dropping = set()
for i in tqdm(range(len(data['substructures']))):
    for j in data['substructures'].iloc[i]:
        try:
            if j != '$':
                if Chem.RemoveHs(Chem.MolFromSmiles(j)).GetNumAtoms() > 30:
                    dropping.add(i)
                if len(data['substructures'].iloc[i]) < 3 and Chem.RemoveHs(Chem.MolFromSmiles(j)).GetNumAtoms() > 25:
                    dropping.add(i)
        except:
            print(i)

In [ ]:
data = data.drop(index = dropping).reset_index(drop = True)

In [ ]:
dropping = []
for i in range(len(data['substructures'])):
    l = 0
    for j in data['substructures'].iloc[i]:
        if j != ['$']:
            l +=1
    if l > 22:
        dropping.append(i)
        
data = data.drop(index = dropping).reset_index(drop = True)

In [ ]:
for i in tqdm(range(len(data['substructures']))):
    for j in data['substructures'].iloc[i]:
        if j != '$':
            if j is None:
                print(i)

In [ ]:
data.to_csv('preprocessed_10m_without_descr.csv')